In [ ]:
import pickle 
import numpy as np
import pandas
import time
import elpigraph
import elpigraphgpu
import rpy2.robjects.packages as rpackages
import rpy2.robjects
import rpy2.robjects.numpy2ri
r_elpigraph = rpackages.importr("ElPiGraph.R")
rpy2.robjects.numpy2ri.activate()

# Quick elpigraph run first to make sure it has been compiled


In [ ]:
tmp = elpigraph.computeElasticPrincipalTree(X = np.random.random(size=(100,10)),
                                                NumNodes = 10,
                                                n_cores = 1,
                                                drawAccuracyComplexity = False, drawEnergy = False,drawPCAView = False,
                                                Do_PCA=False,CenterData=False)
tmp = elpigraphgpu.computeElasticPrincipalTree(X = np.random.random(size=(100,10)),
                                                NumNodes = 10,
                                                n_cores = 1,
                                                drawAccuracyComplexity = False, drawEnergy = False,drawPCAView = False,
                                                Do_PCA=False,CenterData=False)

# Generate plot data
You can modify num_cores as you see fit

In [ ]:
nfeatures = 10
num_points = [10**3,10**4,10**5,10**6]
num_nodes = [10,20,30,40,50,60]#,70,80,90,100]
num_cores = [1,2,4,8,16]#,32,20,24,28,32,36,40,44]

run_points_py_gpu = []
run_points_py = []
run_points_r = []
for npoints in num_points:
    np.random.seed(0)
    data=np.random.random(size=(npoints,nfeatures))

    run_nodes_py_gpu = []
    run_nodes_py = []
    run_nodes_r = []
    for nnode in num_nodes:


        run_cores_py_gpu = []
        run_cores_py = []
        run_cores_r = []
        for ncore in num_cores:
            if ncore == 1:
                # Run Python hybrid
                s = time.time()
                res = elpigraphgpu.computeElasticPrincipalTree(X = data,
                                                              NumNodes = nnode,
                                                              n_cores = 1, #this version can't run in parallel for now
                                                              drawAccuracyComplexity = False, drawEnergy = False,drawPCAView = False,
                                                              Do_PCA=False,CenterData=False)
                end = time.time() - s
                run_cores_py_gpu.append(end)

            # Run Python
            s = time.time()
            res = elpigraph.computeElasticPrincipalTree(X = data,
                                                        NumNodes = nnode,
                                                        n_cores = ncore,
                                                        drawAccuracyComplexity = False, drawEnergy = False,drawPCAView = False,
                                                        Do_PCA=False,CenterData=False)
            end = time.time() - s
            run_cores_py.append(end)


            # Run R
            s = time.time()
            res= r_elpigraph.computeElasticPrincipalTree(X = data,
                                                         NumNodes = nnode,
                                                         n_cores = ncore,
                                                         drawAccuracyComplexity = False, drawEnergy = False,drawPCAView = False,
                                                         Do_PCA=False,CenterData=False)
            end = time.time() - s
            run_cores_r.append(end)


        run_nodes_py_gpu.append(run_cores_py_gpu)
        run_nodes_py.append(run_cores_py)
        run_nodes_r.append(run_cores_r)


    run_points_py_gpu.append(run_nodes_py_gpu)
    run_points_py.append(run_nodes_py)
    run_points_r.append(run_nodes_r)

In [ ]:
with open('3B_run_points_py_gpu.pkl', 'wb') as f:
    pickle.dump(run_points_py_gpu,f)
with open('3B_run_points_py.pkl', 'wb') as f:
    pickle.dump(run_points_py,f)
with open('3B_run_points_r.pkl', 'wb') as f:
    pickle.dump(run_points_r,f)